# Avaliação Final Tecnologias Hacker - Análise de Logs Web

## Objetivo
O objetivo desse projeto é construir um código capaz de analisar se, de acordo com os logs, pode haver uma conexão suspeita no seu servidor web. Para isso, levamos em consideração que os logs disponibilizados estão em formato combined.

In [1]:
#bibliotecas
import re

## Pré Processamento

In [2]:
#Função que le o arquivo e retorna uma lista com as linhas do arquivo
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            return arquivo.readlines()
    except FileNotFoundError:
        print("Arquivo não encontrado")
        return None

In [3]:
lista_linhas_log = ler_arquivo('logs/access.log')
lista_linhas_log[0]

'207.46.13.104 - - [08/Jul/2019:09:07:18 +0200] "GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1" 302 623 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"\n'

In [4]:
# Função de extração das informações
def extrai_info_log(lista_linhas):
    info_logs = {}
    for i in range(len(lista_linhas)):
        dic_log = {}
        info = lista_linhas[i].split()
        dic_log['ip'] = info[0]
        dic_log['id_cliente'] = info[1]
        dic_log['usuario'] = info[2]
        dic_log['data'] = info[3].replace('[','')
        dic_log['requisicao'] = info[5] + " " + info[6] + " " + info[7]
        dic_log['status'] = info[8]
        dic_log['bytes_resposta'] = info[9]
        dic_log['referenciador'] = info[10]
        dic_log['navegador'] = info[11].replace('"','')

        info_logs[i] = dic_log

    return info_logs




In [5]:
dicionariodelogs = extrai_info_log(lista_linhas_log)
dicionariodelogs[0]

{'ip': '207.46.13.104',
 'id_cliente': '-',
 'usuario': '-',
 'data': '08/Jul/2019:09:07:18',
 'requisicao': '"GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1"',
 'status': '302',
 'bytes_resposta': '623',
 'referenciador': '"-"',
 'navegador': 'Mozilla/5.0'}

In [6]:
#Função para separa somente os logs de get
def separa_logs_get(dicionario_logs):
    logs_get = {}
    for i in range(len(dicionario_logs)):
        if 'GET' in dicionario_logs[i]['requisicao']:
            logs_get[i] = dicionario_logs[i]
    return logs_get

#Função que separa os logs de post
def separa_logs_post(dicionario_logs):
    logs_post = {}
    for i in range(len(dicionario_logs)):
        if 'POST' in dicionario_logs[i]['requisicao']:
            logs_post[i] = dicionario_logs[i]
    return logs_post

In [7]:
dicionariodelogs_get = separa_logs_get(dicionariodelogs)
dicionariodelogs_post = separa_logs_post(dicionariodelogs)

### Analisando os logs contendo POST


**coisas suspeitas com POST**:
- post em caminhos suspeitos (/wp-login.php, /xmlrpc.php, ou APIs internas)

- muitas requisições em um curto intervalo

- sql injection (' OR '1'='1, UNION SELECT, ou DROP TABLE.)

- tentativas de login/senhas diferentes: ataque brute force (não sei se o gpt colocou mas eh sobre)

- tentativa de enviar dados com script (script, onload=, ou eval( em campos de texto (indício de XSS)))

- requisições com corpo muito maior que o esperado

- headers http incomuns (X-Forwarded-For alterado ou user-agent indicando automação)

- Arquivos com extensões como .php, .exe, .jsp, ou .sh

- Detecção de endpoints que deveriam ser privados, mas estão sendo acessados publicamente

- Dados contendo comandos como ; ls, && whoami, ou | netstat.

In [ ]:
def caminhos_suspeitos(log_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        # WordPress e CMS
        'wp-login', 'wp-admin', 'xmlrpc.php', 'admin-ajax.php', 
        'wp-content', 'wp-content/plugins/', 'wp-includes/', 'wp-config.php', 'wp-json/',
        # Painéis de Administração
        '/admin', '/administrator', '/console', '/controlpanel', '/login',
        '/dashboard', '/dbadmin', '/config', '/config.php', '/admin.php',
        # Arquivos de Configuração e Backup
        '.env', 'config.php', 'settings.php', 'backup.sql', 'db_backup.sql',
        'db_dump.sql', '.htaccess', '.htpasswd',
        # Caminhos Sensíveis ou de Sistema
        '/etc/passwd', '/etc/shadow', '/proc/self/environ', '/var/www/html',
        # APIs Vulneráveis
        '/api/v1/', '/graphql', '/admin/api/',
        # Exploração de Ferramentas e Bibliotecas
        '/phpmyadmin', '/server-status', '/actuator/health',
        # Frameworks e Ferramentas
        '/debug', '/shell', '/cmd',
        # Padrões de Ataques
        '?file=', '?path=', '?cmd=', '?exec=', '?debug=', '?action=',
        # Explorações de Vulnerabilidades
        'cgi-bin/', 'shell.php', 'eval-stdin.php',
        # Padrões Genéricos de Exploração
        '../', '.php', '.sh', '.exe'
    ]

    for key, value in log_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao']:
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos


In [17]:
logs_post_caminhos_suspeitos = caminhos_suspeitos(dicionariodelogs_post)
#logs_post_caminhos_suspeitos

In [18]:
def checando_sql_injection(logs_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        'select', 'union', 'insert', 'update', 'delete', 'drop', 'alter', 'create',
        'truncate', 'exec', 'grant', 'revoke',
        "' or '1'='1", "--", "#", ";", "/*", "*/",
        "char(", "concat(", "load_file(", "sleep(",
        "information_schema", "table_schema", "column_name",
        "'=", "like '%'", "and 1=1"
    ]
    
    for key, value in logs_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao'].lower():
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos

In [19]:
logs_sql_injection = checando_sql_injection(dicionariodelogs_post)
#logs_sql_injection

In [20]:
def enviar_dados_script(logs_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        '<script>', 'onload=', 'eval(', 
        '<iframe>', 'javascript:', '<embed>', '<object>', '<applet>',
        'onclick=', 'onmouseover=', 'onerror=',
        '%3Cscript%3E', '%3Ciframe%3E',
        'alert(', 'document.cookie', 'window.location', 'innerHTML',
        '"></script>', '<img src="x" onerror=', '"><svg onload=',
        '{{7*7}}', '${7*7}', 
        'base64', '\\u'
    ]

    for key, value in logs_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao'].lower():
                logs_suspeitos[key] = value
                break
    return logs_suspeitos
    

In [22]:
logs_dados_script = enviar_dados_script(dicionariodelogs_post)
#logs_dados_script